# Example 3 - Realtime Synchronization With OSC Timestamps

The third example demonstrates how we can use OSC timestamps to improve synchronization and real-time musical performance over a network. The code is similar to the metronome control system introduced in example 2 but adds the option to adjust the timestamps associated with each OSC packet.

The OSC timestamps are essentially NTP timestamps; a 64-bit fixed floating point number. The first 32 bits specify the number of seconds since midnight on January 1, 1900, and the last 32 bits specify fractional parts of a second to a precision of about 200 picoseconds. Fortunately, to customize the timestamp in python-osc, we only have to provide a floating point number of seconds since the epoch (midnight on January 1, 1900) in UTC, python-osc will do the rest for us. Similarly, in PD, the complexity is hidden as we just need to specify timestampOffset messages to the [mrpeach/packOSC] object to manipulate the timestamp of the next incoming OSC message.


### Files 
1. Notebook "Example 3 - Realtime Synchronization With OSC Timestamps"
1. Notebook "OSC Server"
2. Metronome tick "./tick.wav"
3. Pure Data patch example1.pd from "../pd/example_3_realtime_sync_with_osc_timestamps.pd"
4. Metronome tick "../pd/tick.wav"
5. Notebook "Example 4 - UTC Timestamp Printing"

<p align="left">
 <img src="../../fig/example3.jpg" width=500>
</p>

In [ ]:
from pythonosc import udp_client
from pythonosc import osc_bundle_builder, osc_message_builder
import time
import threading

In [ ]:
# Config
BPM = 100
BEAT_INTERVAL = 60 / BPM

## Define Remote OSC Client (Pure Data)

In [ ]:
clientIp = '127.0.0.1'
clientPort = 8001
client1 = udp_client.UDPClient(clientIp, clientPort)

print(f'Starting client on {clientIp}, port {clientPort}.')

## Message sending logic

In [ ]:
def sendMsg(client):
    print('Sending messages to client.')

    # using the current time once and increment ticks over every while Loop is a more predicable metronome for creating custom timestamp based on a fixed tempo. 
    start_time = time.time()
    beat_number = 0

    while True:
        # Custom timestamp = start time + current while Loop round /tick_numb)
        custom_timestamp = start_time + (beat_number * BEAT_INTERVAL)

        # open a OSC bundle
        # add our timestamp to the bundle builder object
        bundle = osc_bundle_builder.OscBundleBuilder(custom_timestamp)

        # Create and add message
        msg = osc_message_builder.OscMessageBuilder(address="/tick")
        
        msg.add_arg("tick")
        
        bundle.add_content(msg.build())

        # Send the bundle
        client.send(bundle.build())

        # Wait for the next beat
        time.sleep(BEAT_INTERVAL)

## Send Messages to Remote Client

In [ ]:
sendMsg(client1)

## Activity

1. Test and explore the example on your local machine. Try different applications rather than a metronome.